In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [2]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

def findR_S(m_1, m_2, DL = False):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

def findL(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    p_1, p_2 = findp(s)
    # r_cm = findrCM(x_1, x_2, m_1, m_2)
    # L_1 = np.cross(x_1 - r_cm, p_1)
    # L_2 = np.cross(x_2 - r_cm, p_2)
    L_1 = np.cross(x_1, p_1)
    L_2 = np.cross(x_2, p_2)
    return L_1, L_2

def findb(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    p_1, p_2 = findp(s)
    L_1, L_2 = findL(s)
    p_vec = p_1 - p_2
    L_vec = L_1 - L_2
    p = np.linalg.norm(p_vec)
    L = np.linalg.norm(L_vec)
    # b_vec = L_vec/p_vec
    b = L/p
    return b

def findE(s):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    n = r_vec/r
    
    p_1, p_2 = findp(s)
    E_1PN = 1/c**2*(-1/8*np.linalg.norm(p_1)**2/m_1**3  +  1/8*G*m_1*m_2/r*(  -12*np.linalg.norm(p_1)**2/m_1**2  +  14*np.dot(p_1, p_2)/(m_1*m_2)  +  2*np.dot(n, p_1)*np.dot(n, p_2)/(m_1*m_2)  )  +1/4*G*m_1*m_2/r*G*(m_1 + m_2)/r    +     -1/8*np.linalg.norm(p_2)**2/m_2**3  +  1/8*G*m_2*m_1/r*(  -12*np.linalg.norm(p_2)**2/m_2**2  +  14*np.dot(p_2, p_1)/(m_2*m_1)  +  2*np.dot(-n, p_2)*np.dot(-n, p_1)/(m_2*m_1)  )  +1/4*G*m_2*m_1/r*G*(m_2 + m_1)/r  )
    return E_1PN

def γ(v, DL = False):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return 1/np.sqrt(1 - np.dot(v,v)/c**2)

def four_P(p, m, DL = False):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return np.array([np.sqrt(np.dot(p, p)*c**2 + m**2*c**4)/c, p[0], p[1]])


In [3]:
def fourdot(V_1, V_2):
    return V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]

In [4]:
def SItoDL(s, t_max, dt):
    c = const.c.value
    G = const.G.value

    x_1, x_2, v_1, v_2, m_1, m_2 = s
    R_S = findR_S(m_1, m_2, DL = False)

    X_1, X_2 = x_1/R_S, x_2/R_S
    V_1, V_2 = v_1/c, v_2/c
    M_1, M_2 = m_1 * G/(c**2*R_S), m_2 * G/(c**2*R_S)

    S = X_1, X_2, V_1, V_2, M_1, M_2
    T_max, dT = t_max * c/R_S, dt * c/R_S

    return S, T_max, dT, R_S

# def DLtoSI(S, t_max, dt):
#     c = const.c.value
#     G = const.G.value

#     X_1, X_2, V_1, V_2, M_1, M_2 = S
#     R_S = findR_S(m_1, m_2, DL = True)

#     x_1, x_2 = X_1*R_S, X_2*R_S
#     v_1, v_2 = V_1*c, V_2*c
#     m_1, m_2 = M_1 /( G/(c**2*R_S)), M_2 /( G/(c**2*R_S))

#     S = x_1, x_2, v_1, v_2, m_1, m_2
#     t_max, dt = T_max / (c/R_S), dT / (c/R_S)

#     return s, t_max, dt, R_S

In [5]:
def accelerationN_OLD(x_1, x_2, m_1, m_2, DL):
    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])



def accelerationN(x_1, x_2, P_1, P_2, m_1, m_2, DL = False):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value

    def m(i):
        return ms[i-1]

    def p_dot(i, σ):
        result = - G*m_1*m_2/r**3*r_vec[σ]
        if i == 1:
            result = result
        elif i == 2:
            result = - result
        else:
            print('Error: i is different than 1 or 2')
        return result

    def a(i, σ):
        return p_dot(i, σ)/m(i)

    r = rel_len(x_1, x_2)
    r_vec = x_1 - x_2
    r_vec = np.array([0, r_vec[0], r_vec[1]])
    
    ms = np.array([m_1, m_2])

    v_1 = P_1[1:3]/m_1
    v_2 = P_2[1:3]/m_2
    a_1 = np.array([a(1, 1), a(1, 2)])
    a_2 = np.array([a(2, 1), a(2, 2)])
    return v_1, v_2, a_1, a_2


def accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2, DL, α, β):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2
    n = r_vec/r

    a_1 = - β * G*m_2/r**2*n + α * 1/c**2*((5*G**2*m_1*m_2/r**3 + 4*G**2*m_2**2/r**3 + G*m_2/r**2*(3/2*np.dot(n, v_2)**2 - np.dot(v_1, v_1) + 4*np.dot(v_1, v_2) - 2*np.dot(v_2, v_2)))*n + G*m_2/r**2 *(4*np.dot(n, v_1) - 3*np.dot(n, v_2))*v_vec)
    a_2 = - β * G*m_1/r**2*(-n) + α * 1/c**2*((5*G**2*m_2*m_1/r**3 + 4*G**2*m_1**2/r**3 + G*m_1/r**2*(3/2*np.dot((-n), v_1)**2 - np.dot(v_2, v_2) + 4*np.dot(v_2, v_1) - 2*np.dot(v_1, v_1)))*(-n) + G*m_1/r**2 *(4*np.dot((-n), v_2) - 3*np.dot((-n), v_1))*(-v_vec))
    return np.array([a_1, a_2])


def accelerationDatF(x_1, x_2, v_1, v_2, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2

    l = np.cross(r_vec, v_vec)

    a_1 = - G*m_2/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    a_2 =   G*m_1/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    return np.array([a_1, a_2])



def accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    P = np.array([P_1, P_2])
    E_1 = P_1[0]*c
    E_2 = P_2[0]*c
    Es = np.array([E_1, E_2])

    def q_dot(i, σ):
        j = 3 - i
        return p(i, σ)*c**2/E(i) - G/r * (4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))

    def E(i):
        return Es[i-1]

    def E_dot(i):
        return (p(i,1)*p_dot(i,1) + p(i,2)*p_dot(i,2))*c**2/E(i)

    def p(i, σ):
        return P[i-1,σ]

    def p_dot(i, σ):
        result = G*r_vec[σ]/r**3 * (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E_1*E_2)
        if i == 1:
            result = result
        elif i == 2:
            result = - result
        else:
            print('Error: i is different than 1 or 2')
        return result

    def P_dot(i):
        return np.array([E_dot(i)/c, p_dot(i, 1),  p_dot(i, 2)])

    def a(i, σ):
        j = 3 - i
        return p_dot(i, σ)*c**2/E(i) - p(i, σ)*E_dot(i)*c**2/E(i)**2 + G/r**2*r_dot*(4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))        - G/r * (4*((fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2)) - fourdot(P_1, P_2)/(E(1)*E(2))**2*(E_dot(1)*E(2) + E(1)*E_dot(2)))*(E(j)/E(i)*p(i, σ) - p(j, σ)) + 4 * fourdot(P_1, P_2)/(E(1)*E(2)) * ((E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ) - p_dot(j, σ))        - (4* fourdot(P_1, P_2)*(fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2))**2 - 2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**3*(E_dot(1)*E(2) + E(1)*E_dot(2)))*E(j)/E(i)*p(i, σ)*c**2        + (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**2*( (E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ))*c**2)

    r = rel_len(x_1, x_2)
    r_vec = x_1 - x_2
    r_vec = np.array([0, r_vec[0], r_vec[1]])
    r_dot = (r_vec[1]*(q_dot(1, 1) - q_dot(2, 1)) + r_vec[2]*(q_dot(1, 2) - q_dot(2, 2)))/r

    v_1 = np.array([q_dot(1,1), q_dot(1,2)])
    v_2 = np.array([q_dot(2,1), q_dot(2,2)])
    
    a_1 = np.array([a(1, 1), a(1, 2)])
    a_2 = np.array([a(2, 1), a(2, 2)])
    return np.array([v_1, v_2, a_1, a_2])


def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

def punch(v_1, v_2, m_1, m_2, DL):
    p_1 = γ(v_1, DL)*v_1*m_1
    p_2 = γ(v_2, DL)*v_2*m_2
    P_1 = four_P(p_1, m_1, DL)
    P_2 = four_P(p_2, m_2, DL)
    return np.array([P_1, P_2])

In [6]:
def run_model(s, t_max, dt, mode = 'PM', DL = False):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    x_1, x_2, P_1, P_2, m_1, m_2 = s
    a_1, a_2 = 0, 0

    R_S = findR_S(m_1, m_2, DL)

    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findrCM(x_1, x_2, m_1, m_2)
        if mode == 'N':
            v_1, v_2, a_1, a_2 = accelerationN(x_1, x_2, P_1, P_2, m_1, m_2, DL)
            v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
            x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
            P_1, P_2 = np.array([0, v_1[0], v_1[1]])*m_1, np.array([0, v_2[0], v_2[1]])*m_2
        elif mode == 'PM':
            v_1, v_2, a_1, a_2 = accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2, DL)
            v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
            x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
            P_1, P_2 = punch(v_1, v_2, m_1, m_2, DL)
        else:
            print('Error: mode not chosen')
            break
        

        if rel_len(x_1, x_2) <= R_S:
            print('Code interrupted: Radial Dip')
            break
    return pos1, pos2, pos_CM

In [7]:
def format_value(value, decimals):
    """ 
    Checks the type of a variable and formats it accordingly.
    Floats has 'decimals' number of decimals.
    """
    
    if isinstance(value, float):
        return f'{value:.{decimals}e}'
    elif isinstance(value, int):
        return f'{value:d}'
    else:
        return f'{value}'


def values_to_string(values, decimals):
    """ 
    Loops over all elements of 'values' and returns list of strings
    with proper formating according to the function 'format_value'. 
    """
    
    res = []
    for value in values:
        if isinstance(value, list):
            tmp = [format_value(val, decimals) for val in value]
            res.append(f'{tmp[0]} +/- {tmp[1]}')
        else:
            res.append(format_value(value, decimals))
    return res

def len_of_longest_string(s):
    """ Returns the length of the longest string in a list of strings """
    return len(max(s, key=len))

def nice_string_output(d, extra_spacing=5, decimals=3):
    """ 
    Takes a dictionary d consisting of names and values to be properly formatted.
    Makes sure that the distance between the names and the values in the printed
    output has a minimum distance of 'extra_spacing'. One can change the number
    of decimals using the 'decimals' keyword.  
    """
    
    names = d.keys()
    max_names = len_of_longest_string(names)
    
    values = values_to_string(d.values(), decimals=decimals)
    max_values = len_of_longest_string(values)
    
    string = ""
    for name, value in zip(names, values):
        spacing = extra_spacing + max_values + max_names - len(name) - 1 
        string += "{name:s} {value:>{spacing}} \n".format(name=name, value=value, spacing=spacing)
    return string[:-2]


def add_text_to_ax(x_coord, y_coord, string, ax, fontsize=12, color='k'):
    """ Shortcut to add text to an ax with proper font. Relative coords."""
    ax.text(x_coord, y_coord, string, ha='left', va='bottom', family='monospace', fontsize=fontsize,
            transform=ax.transAxes, verticalalignment='top', color=color)
    return None

def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(pos1, pos2, posCM, parameters = np.zeros(8), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:', label = 'CM', lw=0.4)
    ax.plot(x_cm[0], x_cm[1], 'g:', lw=0.5)
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    # ax.plot(x_1[0], x_1[1],'b', label = 'm_1', lw=0.75)
    ax.plot(x_1[0], x_1[1],'b', lw=1)
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    # ax.plot(x_2[0], x_2[1],'r', label = 'm_2', lw=0.75)
    ax.plot(x_2[0], x_2[1],'r', lw=1)
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    d = {
         'E:':      parameters[0],
         '|p_1|:':  parameters[1],
         '|p_2|:':  parameters[2],
         '|L_1|:':  parameters[3],
         '|L_2|:':  parameters[4],
         'R_S:':    parameters[5],
         'b:':      parameters[6],
         'θ:':      parameters[7],
        }

    text = nice_string_output(d, extra_spacing=3, decimals=2)
    add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def orbPlotter_lims(pos1, pos2, posCM, xlim, ylim, parameters = np.zeros(8), aspect = 1, filename='', CM = False, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    # xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    d = {
         'E:':      parameters[0],
         '|p_1|:':  parameters[1],
         '|p_2|:':  parameters[2],
         '|L_1|:':  parameters[3],
         '|L_2|:':  parameters[4],
         'R_S:':    parameters[5],
         'b:':      parameters[6],
         'θ:':      parameters[7],
        }

    text = nice_string_output(d, extra_spacing=3, decimals=2)
    add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def prettyOrbPlotter(pos1, pos2, posCM, parameters, aspect = 1, filename='', CM = False, save = False, show = True, figsize=(11,11)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = 0


    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    # ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    # ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    # ax.set_xlabel('$R_S$', fontsize = 15)
    # ax.set_ylabel('$R_S$', fontsize = 15)
    # ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    # ax.set_title(f'{filename}', fontsize = 20)
    # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
                
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Pretty/{filename}_pretty.png', dpi=300, transparent = False)
    return